In [24]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 500
eval_interval = 50
learning_rate = 3e-4
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 6
n_layer = 2
dropout = 0.2

torch.manual_seed(1337)

In [4]:
text = open("assets/input.txt", "r").read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
# List of all characters in document
chars = sorted(list(set("".join(text))))

# Number of unique characters
vocab_size = len(chars)
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [6]:
# Mapping tables between characters and integers
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

# Take a string, output a list of integers
encode = lambda s: [stoi[c] for c in s]
# Take a list of integers, output a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [25]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, n_embd, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)                                     # (B, T, head_size)
        q = self.query(x)                                   # (B, T, head_size)

        # compute attention scores
        wei = q @ k.transpose(-2,-1)                        # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
        wei /= math.sqrt(k.shape[-1])                       # (B, T, T)
        
        # avoid look-ahead
        tril = torch.tril(torch.ones(T, T))
        wei = wei.masked_fill(tril == 0, float("-inf"))     # (B, T, T)
        wei = F.softmax(wei, dim=-1)                        # (B, T, T)
        wei = self.dropout(wei)
        
        # weighted aggregation of the values
        v = self.value(x)                                   # (B, T, head_size)
        out = wei @ v                                       # (B, T, T) @ (B, T, hs) -> (B, T, head_size)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, head_size):
        super().__init__()
        # best practice to depend head size on n_embed and n_heads
        head_size = n_embd // 4

        self.head1 = Head(n_embd, head_size)
        self.head2 = Head(n_embd, head_size)
        self.head3 = Head(n_embd, head_size)
        self.head4 = Head(n_embd, head_size)

        self.linear = nn.Linear(head_size * 4, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        heads_out = [
            self.head1(x), 
            self.head2(x), 
            self.head3(x), 
            self.head4(x)
        ]
        out = torch.cat(heads_out, dim=-1)
        out = self.linear(out)
        out = self.dropout(out)
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // 4
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding = nn.Embedding(block_size, n_embd)
        
        # put one block after the other sequentially (not parallel like multi-head attention)
        self.blocks = nn.Sequential(*[Block(n_embd) for _ in range(n_layer)])
        
        # final layer norm
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):

        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding(idx)                 # (B, T, C)
        pos_emb = self.pos_embedding(torch.arange(T))       # (T, C)
        x = tok_emb + pos_emb                               # (B, T, C)
        x = self.blocks(x)                                  # (B, T, C)
        x = self.ln_f(x)                                    # (B, T, C)
        logits = self.lm_head(x)                            # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [26]:

model = GPTLanguageModel()
# m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

0.029761 M parameters
step 0: train loss 4.1696, val loss 4.1712
step 50: train loss 3.8083, val loss 3.8115
step 100: train loss 3.5407, val loss 3.5839
step 150: train loss 3.3845, val loss 3.4159
step 200: train loss 3.2517, val loss 3.2716
step 250: train loss 3.1420, val loss 3.1673
step 300: train loss 3.0747, val loss 3.1191
step 350: train loss 3.0320, val loss 3.0509
step 400: train loss 2.9971, val loss 3.0014
step 450: train loss 2.8894, val loss 2.9236
step 499: train loss 2.8691, val loss 2.8869


 !

AR wx$iw,?eeap, ras autpceceirth ,laud lamhe Ads w hhusOntiaplud. etghiteluney I
:nshe Ay
Seus le,
G t ge.f;ytssit pt neomitowe y:

$W acohoiD: ,va'eL
EOUKMRNY:iry,o the shi&a-;
IlacC iL;R
Eish eyesot at whe nn otabildaepes He&iuniloldme pn tomnGanouIf X: irLLla,
Wvniw  ndfotr hDIeomore rpint noe veuaWAv;
?o o n  s he bha;Ad r  hee irte,WNl ath:D'Iphy.,SWsorhheromeomta : .:enrasar yre fourauHe tedlHitm XoaNdretVa Mad conr se t  y  panUI wer de oeics,,,o ac tse ,LAcekel Paisud